In [1]:
import requests
import json
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [2]:
url = "http://scholarworks.unist.ac.kr/browse-researcher"
data = {
    "sortType": "English",
    "page": 1,
    "offset": 0
}

In [3]:
## 데이터 정보를 보내지 않았다!
r = requests.get(url, params=data)
text = r.text
soup = BeautifulSoup(text, 'lxml')

In [4]:
li = soup.findAll("li")[16:76]
li[0:5]

[<li>
 <a href="researcher-profile?ep=2741">Ahn, Sangjoon
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2781">Amblard, Francois
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2641">An, Kwangjin
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=1881">Au, Tsz-Chiu
 
 						</a>
 </li>, <li>
 <a href="researcher-profile?ep=2041">Bae, Hantaek
 
 						</a>
 </li>]

In [5]:
link = []
name = []

def makelink(pg):
    url = "http://scholarworks.unist.ac.kr/browse-researcher"
    data = {
        "sortType": "English",
        "page": pg,
        "offset": 0
    }
    r = requests.get(url, params=data)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    for i in soup.findAll("li")[16:76]:
        link.append(i.find("a")['href'])
        name.append(i.getText().replace("\n","").replace("\r","").replace("\t",""))

In [6]:
for i in range(5):
    makelink(i+1)

In [7]:
def makelink6():
    url = "http://scholarworks.unist.ac.kr/browse-researcher"
    data = {
        "sortType": "English",
        "page": 6,
        "offset": 0
    }
    r = requests.get(url, params=data)
    text = r.text
    soup = BeautifulSoup(text, 'lxml')
    for i in soup.findAll("li")[16:28]:
        link.append(i.find("a")['href'])
        name.append(i.getText().replace("\n","").replace("\r","").replace("\t",""))

In [8]:
makelink6()

In [9]:
link[311], name[311]

('researcher-profile?ep=1302', 'Zhao, Bum Suk')

In [10]:
client = MongoClient()
db = client.aboutdepart

In [11]:
for i, k in zip(link,name):    
    document = {}
    document["link"] = i
    document["name"] = k
    db.scholar.insert_one(document)

In [12]:
db.scholar.count()

/Users/seungheondoh/anaconda3/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


312

In [14]:
Contents = db.scholar.find()
Contents[5]

{'_id': ObjectId('5be8188b20c66c2c9ea60082'),
 'link': 'researcher-profile?ep=1249',
 'name': 'Bae, Joonbum'}

-------------------------------------

### db.scholar에서 저장한 link를 타고 url을 호출합니다.

In [87]:
url = "http://scholarworks.unist.ac.kr/"+str(link[100])
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

In [88]:
soup

<!DOCTYPE html>
<html>
<head>
<title>ScholarWorks: Researcher Page</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="DSpace 4.3" name="Generator"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="IE=Edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<!-- Facebook 공유 Thumbnail -->
<meta content="http://scholarworks.unist.ac.kr:80/image/background/facebook-thumb800.jpg" property="og:image"/>
<link href="http://scholarworks.unist.ac.kr:80/image/background/facebook-thumb800.jpg" rel="image_src"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/css/common.css" media="screen" rel="stylesheet" title="web2.0" type="text/css"/>
<script src="/static/js/jquery/jquery-1.10.2.min.js" type="text/javascript"></script>
<script src="/js/utils.js" type="text/javascript"></script>
<script src="/js/koasas.js" type=

In [89]:
soup.find('div',{'class' : "research_info_view"})

<div class="research_info_view">
<div class="researcher_idbox">
<ul>
<li>
<a href="http://www.researcherid.com/rid/H-4798-2016" id="H-4798-2016" target="_blank"><img alt="ResearcherID" class="tooltip" src="/image/common/research_icon02.gif" title="ResearcherID"/></a>
</li>
<li>
<a href="http://www.scopus.com/authid/detail.url?authorId=15123900700" id="http://www.scopus.com/authid/detail.url?authorId=15123900700" target="_blank"><img alt="Scopus" class="tooltip" src="/image/common/SCPID_icon.png" title="Scopus" width="34"/></a>
</li>
</ul>
</div>
<p class="neme_p">
			Kee, Jung-Min (기정민)
			
		</p>
<dl>
<dt>Department</dt>
<dd>
			School of Natural Science(자연과학부)
			</dd>
</dl>
<dl>
<dt>Research Interests</dt>
<dd>
			Chemical biology, organic synthesis, peptide chemistry, synthetic protein chemistry
       		</dd>
</dl>
<dl>
<dt>Lab</dt>
<dd>Bioorganic and Chembio Lab</dd>
</dl>
<dl>
<dt>Website</dt>
<dd><a href="http://kee-lab.org" target="_blank">http://kee-lab.org</a></dd>
</dl>
</d

In [90]:
soup.find('div',{'class' : "research_info_view"}).findAll('dl')

[<dl>
 <dt>Department</dt>
 <dd>
 			School of Natural Science(자연과학부)
 			</dd>
 </dl>, <dl>
 <dt>Research Interests</dt>
 <dd>
 			Chemical biology, organic synthesis, peptide chemistry, synthetic protein chemistry
        		</dd>
 </dl>, <dl>
 <dt>Lab</dt>
 <dd>Bioorganic and Chembio Lab</dd>
 </dl>, <dl>
 <dt>Website</dt>
 <dd><a href="http://kee-lab.org" target="_blank">http://kee-lab.org</a></dd>
 </dl>]

In [91]:
#department
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[0].find('dd').getText().replace("\r","").replace("\t","").replace("\n","")

'School of Natural Science(자연과학부)'

In [92]:
#Research Interest
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[1].find('dd').getText().replace("\r","").replace("\t","").replace("\n","")

'Chemical biology, organic synthesis, peptide chemistry, synthetic protein chemistry       '

In [93]:
#Lab
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[2].find('dd').getText()

'Bioorganic and Chembio Lab'

In [94]:
#Website
soup.find('div',{'class' : "research_info_view"}).findAll('dl')[3].find('dd').getText()

'http://kee-lab.org'

### 학술지 정보

In [95]:
soup.find('div',{'class' : "researcher_idbox"})

<div class="researcher_idbox">
<ul>
<li>
<a href="http://www.researcherid.com/rid/H-4798-2016" id="H-4798-2016" target="_blank"><img alt="ResearcherID" class="tooltip" src="/image/common/research_icon02.gif" title="ResearcherID"/></a>
</li>
<li>
<a href="http://www.scopus.com/authid/detail.url?authorId=15123900700" id="http://www.scopus.com/authid/detail.url?authorId=15123900700" target="_blank"><img alt="Scopus" class="tooltip" src="/image/common/SCPID_icon.png" title="Scopus" width="34"/></a>
</li>
</ul>
</div>

In [96]:
for i in soup.find('div',{'class' : "researcher_idbox"}).findAll('a'):
    if "http://www.scopus.com/authid/detail.url?" in i['href']:
        print(i['href'].replace("http://www.scopus.com/authid/detail.url?authorId=",""))

15123900700


## Final
db 스키마

- id : 스코퍼스 고유 id
- department : 소속학과
- ResearchInterest : 관심단어
- Lab : 연구실 명
- Website : 웹사이트 페이지

In [100]:
def savedata(a):
    url = "http://scholarworks.unist.ac.kr/"+str(a)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    dl = soup.find('div',{'class' : "research_info_view"}).findAll('dl')
    document ={}
    try:
        for i in soup.find('div',{'class' : "researcher_idbox"}).findAll('a'):
            if "http://www.scopus.com/authid/detail.url?" in i['href']:
                document['id']= i['href'].replace("http://www.scopus.com/authid/detail.url?authorId=","")
    except:
        document['id'] = ""
    document['department']= dl[0].find('dd').getText().replace("\r","").replace("\t","").replace("\n","")
    document['ResearchInterest']=dl[1].find('dd').getText().replace("\r","").replace("\t","").replace("\n","")
    document['Lab']=dl[2].find('dd').getText()
    document['Website']=dl[3].find('dd').getText()
    db.profdata.insert_one(document)

In [101]:
for i in link:
    savedata(i)

In [103]:
k = db.profdata.find()
for i in k[100:110]:
    print(i)

{'_id': ObjectId('5be82c2d20c66c2c9ea604b3'), 'id': '15123900700', 'department': 'School of Natural Science(자연과학부)', 'ResearchInterest': 'Chemical biology, organic synthesis, peptide chemistry, synthetic protein chemistry       ', 'Lab': 'Bioorganic and Chembio Lab', 'Website': 'http://kee-lab.org'}
{'_id': ObjectId('5be82c2f20c66c2c9ea604b4'), 'id': '7003629876', 'department': 'School of Mechanical, Aerospace and Nuclear Engineering(기계항공및원자력공학부)', 'ResearchInterest': 'Laser materials processing, Laser material interaction       ', 'Lab': 'Laser Processing & Multiphysics Systems Laboratory', 'Website': 'http://ptlresearch.com/'}
{'_id': ObjectId('5be82c3120c66c2c9ea604b5'), 'department': 'School of Natural Science(자연과학부)', 'ResearchInterest': '', 'Lab': '', 'Website': ''}
{'_id': ObjectId('5be82c3320c66c2c9ea604b6'), 'id': '35264258100', 'department': 'School of Natural Science(자연과학부)', 'ResearchInterest': 'Carbon materials, polymer, Layer-by-Layer (LbL) assembly, hyperbranched polymer